# target_encoding

In [51]:
!pip install xgboost

     |████████████████████████████████| 1.2 MB 953 kB/s eta 0:00:01


In [52]:
#適当にモジュールをインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from scipy import stats
from scipy.stats import rankdata, norm

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import *

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
import time, os, warnings, random, string, re, gc, sys

import category_encoders as ce

import lightgbm as lgb
import catboost as cb

from xgboost import XGBRegressor


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/naokitakahashi/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/naokitakahashi/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [19]:
train_df = pd.read_csv('../data/original/train.csv')
test_df = pd.read_csv('../data/original/test.csv')

In [3]:
# カテゴリ変数と数値変数を抜き出せるようにする
cat_features = ["cat" + str(i) for i in range(10)]
num_features = ["cont" + str(i) for i in range(14)]

In [59]:
#コピーをとっておく
tr_orig = train_df.copy()
ts_orig = test_df.copy()

In [30]:
to_drop = tr_orig[tr_orig["target"] <= 6].index
tr_orig = tr_orig.drop(to_drop)

In [7]:
train = tr_orig.copy()
test = ts_orig.copy()
target = train.pop('target')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299628 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      299628 non-null  int64  
 1   cat0    299628 non-null  object 
 2   cat1    299628 non-null  object 
 3   cat2    299628 non-null  object 
 4   cat3    299628 non-null  object 
 5   cat4    299628 non-null  object 
 6   cat5    299628 non-null  object 
 7   cat6    299628 non-null  object 
 8   cat7    299628 non-null  object 
 9   cat8    299628 non-null  object 
 10  cat9    299628 non-null  object 
 11  cont0   299628 non-null  float64
 12  cont1   299628 non-null  float64
 13  cont2   299628 non-null  float64
 14  cont3   299628 non-null  float64
 15  cont4   299628 non-null  float64
 16  cont5   299628 non-null  float64
 17  cont6   299628 non-null  float64
 18  cont7   299628 non-null  float64
 19  cont8   299628 non-null  float64
 20  cont9   299628 non-null  float64
 21  cont10  29

## target_encoding

In [40]:
# とりあえず全てを学習データとする
# 変数をループしてtarget encoding
for c in cat_features:
    # 学習データ全体で各カテゴリにおけるtargetの平均を計算
    data_tmp = pd.DataFrame({c: tr_orig[c], 'target': tr_orig["target"]})
    target_mean = data_tmp.groupby(c)['target'].mean()
    # テストデータのカテゴリを置換
    ts_orig[c] = ts_orig[c].map(target_mean)

    # 学習データの変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan, tr_orig.shape[0])

    # 学習データを分割
    kf = KFold(n_splits=4, shuffle=True, random_state=72)
    for idx_1, idx_2 in kf.split(tr_orig):
        # out-of-foldで各カテゴリにおける目的変数の平均を計算
        target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
        # 変換後の値を一時配列に格納
        tmp[idx_2] = tr_orig[c].iloc[idx_2].map(target_mean)

    # 変換後のデータで元の変数を置換
    tr_orig[c] = tmp

In [31]:
data_tmp = pd.DataFrame({"cat0": tr_orig["cat0"], 'target': tr_orig["target"]})

In [32]:
target_mean = data_tmp.groupby("cat0")['target'].mean()

In [33]:
target_mean

cat0
A    8.243577
B    8.251040
Name: target, dtype: float64

In [34]:
tmp = np.repeat(np.nan, tr_orig.shape[0])

In [35]:
len(tmp)

299628

In [36]:
kf = KFold(n_splits=4, shuffle=True, random_state=72)

In [39]:
for idx_1, idx_2 in kf.split(tr_orig):
#     # out-of-foldで各カテゴリにおける目的変数の平均を計算
#     target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
#     # 変換後の値を一時配列に格納
#     tmp[idx_2] = tr_orig[c].iloc[idx_2].map(target_mean)

[     3      5      7 ... 299618 299621 299627]
[    10     11     14 ... 299617 299619 299625]
[     0      1      2 ... 299608 299613 299623]
[     4      6      8 ... 299622 299624 299626]


In [47]:
X = tr_orig.drop(["id", "target"], axis=1)
X_test = ts_orig.drop(["id"], axis=1)
y = tr_orig["target"]

In [53]:
rg = Ridge(alpha=0.1, random_state=2021)

In [54]:
rg.fit(X, y)

Ridge(alpha=0.1, random_state=2021)

In [55]:
y_test = rg.predict(X_test)

In [56]:
y_test[:5]

array([8.05828469, 8.28070063, 8.30691994, 8.16474993, 8.19180309])

In [57]:
submission = pd.read_csv("../data/original/sample_submission.csv")
submission.target = y_test
submission.to_csv("../data/submit/submission_4.csv", index=False)

In [58]:
submission.head()

,id,target
0,0,8.058285
1,5,8.280701
2,15,8.306920
3,16,8.164750
4,17,8.191803


提出したところ、スコアは0.73984であった。
手元でこのスコアに近いスコアが出せるかどうか交差検証してみる

In [61]:
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for i, (tr_idx, va_idx) in enumerate(kf.split(tr_orig)):

    # 学習データからバリデーションデータを分ける
    tr_x, va_x = tr_orig.iloc[tr_idx].copy(), tr_orig.iloc[va_idx].copy()
    tr_y, va_y = tr_orig["target"].iloc[tr_idx], tr_orig["target"].iloc[va_idx]

    # 変数をループしてtarget encoding
    for c in cat_features:
        # 学習データ全体で各カテゴリにおけるtargetの平均を計算
        data_tmp = pd.DataFrame({c: tr_x[c], 'target': tr_y})
        target_mean = data_tmp.groupby(c)['target'].mean()
        # バリデーションデータのカテゴリを置換
        va_x.loc[:, c] = va_x[c].map(target_mean)

        # 学習データの変換後の値を格納する配列を準備
        tmp = np.repeat(np.nan, tr_x.shape[0])
        kf_encoding = KFold(n_splits=4, shuffle=True, random_state=72)
        for idx_1, idx_2 in kf_encoding.split(tr_x):
            # out-of-foldで各カテゴリにおける目的変数の平均を計算
            target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
            # 変換後の値を一時配列に格納
            tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)

        tr_x.loc[:, c] = tmp

In [63]:
va_x.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,8.248903,8.203964,8.224976,8.237502,8.241302,8.229732,8.241855,8.240892,8.283497,8.254729,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
3,4,8.248903,8.203964,8.245626,8.237502,8.241302,8.252869,8.241855,8.240892,8.283497,8.270896,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,8.238862,8.278536,8.245626,8.237502,8.241302,8.252869,8.241855,8.240892,8.229450,8.254729,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260
5,7,8.238862,8.203964,8.245626,8.237502,8.241302,8.252869,8.241855,8.240892,8.256872,8.261746,0.402056,0.837058,0.737991,0.778429,0.530250,0.392432,0.658169,0.997473,0.569874,0.960864,0.238050,0.316065,0.731729,0.694719,8.028558
10,12,8.248903,8.278536,8.245626,8.237502,8.241302,8.252869,8.241855,8.240892,8.229450,8.223311,0.797276,0.429131,0.285103,0.488405,0.287287,0.650796,0.706531,0.781168,0.828092,0.817323,0.495890,0.713362,0.743404,0.437057,8.078226


In [65]:
X = tr_x.drop(["id", "target"], axis=1)
# X_test = ts_orig.drop(["id"], axis=1)
y = tr_y.copy()

In [66]:
rg.fit(X, y)

Ridge(alpha=0.1, random_state=2021)

In [69]:
pr_y = rg.predict(va_x.drop(["id", "target"], axis=1))

In [70]:
y_pr = rg.predict(tr_x.drop(["id", "target"], axis=1))

In [71]:
print('train rmse:' , np.sqrt(mean_squared_error(y, y_pr)),'val rmse:' , np.sqrt(mean_squared_error(va_y, pr_y)))

train rmse: 0.7381813914833842 val rmse: 0.7389147654997751


訓練データのRMSEと評価データのRMSEには大差はない
ただし、提出データとは0.001のごさがあった（まあ、こんなものなのだろう）

## LightGBM
どちらが良いか比較するためにLightGBMでもスコアを見てみる

In [72]:
SEED = 8970365

In [73]:
params_lgb = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.003899156646724397,
    "num_leaves": 63,
    "max_depth": 99,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.8805303688019942,
    "min_child_samples": 290,
    "reg_alpha": 9.562925363678952,
    "reg_lambda": 9.355810045480153,
    "max_bin": 882,
    "min_data_per_group": 127,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 19,
    "verbosity": -1,
    "bagging_seed": SEED,
    "feature_fraction_seed": SEED,
    "seed": SEED
}

In [74]:
lgb_train = lgb.Dataset(tr_x.drop(["id", "target"],axis=1), tr_y)
lgb_val = lgb.Dataset(va_x.drop(["id", "target"],axis=1), va_y)

In [75]:
model_lgb = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      num_boost_round=100000,
                      early_stopping_rounds=200,
                      verbose_eval=1000)

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.72975
[2000]	valid_0's rmse: 0.7237
[3000]	valid_0's rmse: 0.720942
[4000]	valid_0's rmse: 0.719506
[5000]	valid_0's rmse: 0.718711
[6000]	valid_0's rmse: 0.718292
[7000]	valid_0's rmse: 0.718059
[8000]	valid_0's rmse: 0.717916
[9000]	valid_0's rmse: 0.717848
[10000]	valid_0's rmse: 0.717807
Early stopping, best iteration is:
[10266]	valid_0's rmse: 0.717796


訓練データは見られないが、RMSEの値はリッジ回帰よりも良い。

In [76]:
pred_lgb = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)

In [77]:
pred_lgb[:5]

array([8.04691798, 8.33734582, 8.30845009, 8.49216924, 8.17268597])

In [79]:
submission = pd.read_csv("../data/original/sample_submission.csv")
submission.target = y_test
submission.to_csv("../data/submit/005_submission.csv", index=False)

提出してみたところ、スコアは0.73984であった。
何と前回と同じである。まるで意味がわからない。